# Task-2
### CNN Network in MNIST

In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

## 1. Define Network

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.conv2 = nn.Conv2d(10, 35, 3)
        self.fc1 = nn.Linear(35 * 5 * 5, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1, 35*5*5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## 2. Define training loop

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if (batch_idx + 1) % len(train_loader) == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx , len(train_loader),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Accuracy: {}/{} ({:.2f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## 3. Model parameters

In [12]:
batch_size = 1024
epochs = 10
lr = 0.01

seed = 1
device='cuda'

## 4. Define model

In [13]:
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

## 5. Build train / test dataloaders

In [7]:
dataset_train = datasets.MNIST('./data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))

dataset_test = datasets.MNIST('./data', train=False, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

## 6. Training

In [16]:
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [58/59 (98%)]	Loss: 0.009671
Test set: Accuracy: 9898/10000 (98.98%)

Train Epoch: 2 [58/59 (98%)]	Loss: 0.005413
Test set: Accuracy: 9905/10000 (99.05%)

Train Epoch: 3 [58/59 (98%)]	Loss: 0.004617
Test set: Accuracy: 9891/10000 (98.91%)

Train Epoch: 4 [58/59 (98%)]	Loss: 0.006089
Test set: Accuracy: 9892/10000 (98.92%)

Train Epoch: 5 [58/59 (98%)]	Loss: 0.017811
Test set: Accuracy: 9890/10000 (98.90%)

Train Epoch: 6 [58/59 (98%)]	Loss: 0.007367
Test set: Accuracy: 9893/10000 (98.93%)

Train Epoch: 7 [58/59 (98%)]	Loss: 0.001014
Test set: Accuracy: 9889/10000 (98.89%)

Train Epoch: 8 [58/59 (98%)]	Loss: 0.000972
Test set: Accuracy: 9905/10000 (99.05%)

Train Epoch: 9 [58/59 (98%)]	Loss: 0.004882
Test set: Accuracy: 9897/10000 (98.97%)

Train Epoch: 10 [58/59 (98%)]	Loss: 0.007279
Test set: Accuracy: 9914/10000 (99.14%)



## 7. Results

https://docs.google.com/spreadsheets/d/1QjyQDkaD11UtiPWjT5aQKG5mX5eC6Yf9o2NdkGmOMos/edit?usp=sharing

|  optimizer | epochs_max | layers | batch_size | best_acc |
| --- | --- | --- | --- | --- |
|  ADAM (0.01) | 10 | conv(10x5x5) -> relu -> maxpool(2x2) -> conv(30x3x3) > relu -> maxpool(2x2) -> fc(100) -> fc(10) | 16 | 97,03 |
|  ADAM (0.01) | 10 | conv(10x5x5) -> relu -> maxpool(2x2) -> conv(30x3x3) > relu -> maxpool(2x2) -> fc(100) -> fc(10) | 32 | 98,11 |
|  ADAM (0.01) | 10 | conv(10x5x5) -> relu -> maxpool(2x2) -> conv(30x3x3) > relu -> maxpool(2x2) -> fc(100) -> fc(10) | 64 | 98,55 |
|  ADAM (0.01) | 10 | conv(10x5x5) -> relu -> maxpool(2x2) -> conv(30x3x3) > relu -> maxpool(2x2) -> fc(100) -> fc(10) | 128 | 98,87 |
|  ADAM (0.01) | 10 | conv(10x5x5) -> relu -> maxpool(2x2) -> conv(30x3x3) > relu -> maxpool(2x2) -> fc(100) -> fc(10) | 256 | 99,05 |
|  ADAM (0.01) | 10 | conv(10x5x5) -> relu -> maxpool(2x2) -> conv(30x3x3) > relu -> maxpool(2x2) -> fc(100) -> fc(10) | 512 | 99,06 |
|  ADAM (0.01) | 10 | conv(10x5x5) -> relu -> maxpool(2x2) -> conv(30x3x3) > relu -> maxpool(2x2) -> fc(100) -> fc(10) | 1024 | 99,23 |
|  SGD (0.01) | 250 | conv(10x5x5) -> relu -> maxpool(2x2) -> conv(30x3x3) > relu -> maxpool(2x2) -> fc(100) -> fc(10) | 1024 | 98,23 |
|  SGD (0.001) start from exp000 | 30 | conv(10x5x5) -> relu -> maxpool(2x2) -> conv(30x3x3) > relu -> maxpool(2x2) -> fc(100) -> fc(10) | 1024 | 99,23 |
|  SGD (0.0001) start from exp000 | 30 | conv(10x5x5) -> relu -> maxpool(2x2) -> conv(30x3x3) > relu -> maxpool(2x2) -> fc(100) -> fc(10) | 1024 | **99,23** |